In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf

In [2]:
columnNames = [
    'Sample code number',
    'Clump Thickness',
    'Uniformity of Cell Size',
    'Uniformity of Cell Shape',
    'Marginal Adhesion',
    'Single Epithelial Cell Size',
    'Bare Nuclei',
    'Bland Chromatin',
    'Normal Nucleoli',
    'Mitoses',
    'Class'
]
data = pd.read_csv('breast-cancer-wisconsin.data', names = columnNames)
print(data.shape)
data.head(10)

(699, 11)


,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2


In [3]:
#Replace ？ with NaN
data = data.replace(to_replace='?',value = np.nan)
#Discard data with missing values (discard as long as one dimension is missing)
data = data.dropna(how='any')
data.shape

(683, 11)

In [4]:
data_shuffled = shuffle(data,random_state = 17).reset_index(drop=True)
data_shuffled.head(10)

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1231853,4,2,2,1,2,1,2,1,1,2
1,1304595,3,1,1,1,1,1,2,1,1,2
2,1083817,3,1,1,1,2,1,2,1,1,2
3,1175937,5,4,6,7,9,7,8,10,1,4
4,188336,5,3,2,8,5,10,8,1,2,4
5,1080233,7,6,6,3,2,10,7,1,1,4
6,1232225,10,4,5,5,5,10,4,1,1,4
7,1230175,10,10,10,3,10,10,9,10,1,4
8,1212422,3,1,1,1,2,1,3,1,1,2
9,1243256,10,4,3,2,3,10,5,3,2,4


In [23]:
data, label =data_shuffled[columnNames[1:10]], data_shuffled[columnNames[10]]
data['Extra features'] = 1
data =data.astype('int')
data_list = data.values.tolist()
label =label.astype('int')
label_list = label.values.tolist()
print(f'data size is {data.shape}')
print(f'label size is {label.shape}')

data size is (683, 10)
label size is (683,)


C:\Users\Alienware\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
def Scale(data):
    arr =  np.array(data)
    new_data = []
    for i in range(len(arr)):
        tmp = []
        for j in range(len(arr[i])):
            tmp.append(2*((arr[i][j]-np.min(arr[i])+1E-6)/(np.max(arr[i])-np.min(arr[i])+1E-6))-1)
        new_data.append(tmp)
    return new_data
def Onehot(label,num):
    length = len(label)
    t_nk = np.zeros((length,num),dtype='int')
    for i in range(length):
        if label[i] == 2:
            t_nk[i][0] = 1
        elif label[i] == 4:
            t_nk[i][1] = 1
    return t_nk

In [27]:
# data = Scale(data)
print(data[:2])

[[1.0, -0.3333328888890371, -0.3333328888890371, -0.9999993333335555, -0.3333328888890371, -0.9999993333335555, -0.3333328888890371, -0.9999993333335555, -0.9999993333335555, -0.9999993333335555], [1.0, -0.9999990000005, -0.9999990000005, -0.9999990000005, -0.9999990000005, -0.9999990000005, 4.999997498256192e-07, -0.9999990000005, -0.9999990000005, -0.9999990000005]]


In [28]:
# label = Onehot(label, 2)
print(label[:5])

[[1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]]


In [30]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
data_train_index,data_test_index = [],[]
for train_index, test_index in kf.split(data):
    data_train_index.append(train_index)
    data_test_index.append(test_index)
data_train, label_train = [], []
data_test,label_test = [],[]
for i in range(5):
    temp = []
    for j in range(len(data_train_index[i])):
        temp.append(data[data_train_index[i][j]])
    data_train.append(temp)
    temp = []
    for j in range(len(data_train_index[i])):
        temp.append(label[data_train_index[i][j]])
    label_train.append(temp)

    temp = []
    for k in range(len(data_test_index[i])):
        temp.append(data[data_test_index[i][k]])
    data_test.append(temp)
    temp = []
    for k in range(len(data_test_index[i])):
        temp.append(label[data_test_index[i][k]])
    label_test.append(temp)
for i in range(5):
    print(data_train[i][-5:])
#     print(data_test[i][-5:])
    print(label_train[i][-5:])

[[1.0, 1.0, 1.0, 0.33333340740739925, 1.0, 1.0, 0.5555556049382662, -0.7777775802469356, -0.9999997777778025, -0.9999997777778025], [-0.9999990000005, 4.999997498256192e-07, -0.9999990000005, 1.0, 4.999997498256192e-07, -0.9999990000005, -0.9999990000005, 4.999997498256192e-07, -0.9999990000005, -0.9999990000005], [-0.9999990000005, -0.9999990000005, -0.9999990000005, -0.9999990000005, 1.0, 4.999997498256192e-07, 4.999997498256192e-07, -0.9999990000005, -0.9999990000005, -0.9999990000005], [0.20000015999996812, -0.99999960000008, -0.99999960000008, -0.99999960000008, -0.599999680000064, -0.99999960000008, -0.19999976000004793, 1.0, -0.99999960000008, -0.99999960000008], [1.0, -0.999999500000125, -0.999999500000125, 2.499999376404105e-07, -0.49999962500009376, -0.999999500000125, -0.999999500000125, -0.999999500000125, -0.999999500000125, -0.999999500000125]]
[array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0])]
[[1.0, 1.0, 1.0, 0.33333340740739925, 1.0, 1.0, 0.555

In [96]:
x = tf.placeholder(tf.float64, [None, 10], name="input_x")
label = tf.placeholder(tf.float64, [None, 2], name="input_y")

W1 = tf.Variable((np.random.random([10, 50])*2 - 1)*np.sqrt(6/(13)))
W2 = tf.Variable((np.random.random([50, 2])*2 - 1)*np.sqrt(6/(13)))

h = tf.matmul(x, W1)
h = tf.nn.tanh(h) # 非线性特征
y = tf.matmul(h, W2)

loss = tf.reduce_mean(tf.square(y-label))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [97]:
for i in range(5):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    Acc = []
    for itr in range(2001):
        sess.run(train_step, feed_dict={x: data_train[i], label: label_train[i]})
        if itr % 200 == 0:
            print("group%1d"%(i+1),"step:%6d  train accuracy:"%itr, sess.run(accuracy, feed_dict={x: data_train[i],
                                            label: label_train[i]}),
                  "test accuracy:",sess.run(accuracy, feed_dict={x: data_test[i], label: label_test[i]}))
    Acc.append(sess.run(accuracy, feed_dict={x: data_test[i],
                                            label: label_test[i]}))

group1 step:     0  train accuracy: 0.8223443 test accuracy: 0.8029197
group1 step:   200  train accuracy: 0.93406594 test accuracy: 0.9270073
group1 step:   400  train accuracy: 0.94139194 test accuracy: 0.9489051
group1 step:   600  train accuracy: 0.9542124 test accuracy: 0.93430656
group1 step:   800  train accuracy: 0.95787543 test accuracy: 0.9489051
group1 step:  1000  train accuracy: 0.9542124 test accuracy: 0.9489051
group1 step:  1200  train accuracy: 0.95604396 test accuracy: 0.9489051
group1 step:  1400  train accuracy: 0.95604396 test accuracy: 0.9489051
group1 step:  1600  train accuracy: 0.96153843 test accuracy: 0.94160587
group1 step:  1800  train accuracy: 0.96153843 test accuracy: 0.94160587
group1 step:  2000  train accuracy: 0.96153843 test accuracy: 0.93430656
group2 step:     0  train accuracy: 0.8131868 test accuracy: 0.82481754
group2 step:   200  train accuracy: 0.93406594 test accuracy: 0.93430656
group2 step:   400  train accuracy: 0.94505495 test accuracy: 

In [98]:
print(np.array(Acc).mean())

0.9338235
